# Specific to apartment units - NY

In [ ]:
#!pip install snowflake-connector-python
#!pip install snowflake-connector-python[pandas]
#!pip install usaddress
import snowflake.connector
from snowflake.connector import cursor
import usaddress
import re

In [ ]:
ctx = snowflake.connector.connect(
     user = '',
     password = '',
     account = ''
 )

cursor = ctx.cursor()

In [ ]:
import pandas as pd
sql_MLS = """"""
df_MLS = pd.read_sql(sql_MLS, con=ctx)

In [ ]:
sql_DATATREE = """"""
df_DATATREE = pd.read_sql(sql_DATATREE, con=ctx)

In [ ]:
df_DATATREE = df_DATATREE.drop_duplicates(['SITUSFULLSTREETADDRESS',
                            'SITUSZIP5',
                            'SITUSSTATE',
                            'SITUSCITY'])
df_MLS = df_MLS.drop_duplicates(['CC_PROPERTY_ADDR_DISPLAY_1',
                                'CC_PROPERTY_ADDR_POSTAL_CODE',
                                'CC_PROPERTY_ADDR_CITY',
                                'CC_PROPERTY_ADDR_STATE'])

In [ ]:
from string import punctuation
import re
# punctuation -> space
remove_punctiuation = str.maketrans(punctuation, ' '*len(punctuation))

df_MLS['MLS_ADDRESS_clean'] = df_MLS['CC_PROPERTY_ADDR_DISPLAY_1'].apply(lambda x: x.translate(remove_punctiuation))
df_DATATREE['DATATREE_ADDRESS_clean'] = df_DATATREE['SITUSFULLSTREETADDRESS'].apply(lambda x: x.translate(remove_punctiuation))
# several spaces -> one space
df_MLS['MLS_ADDRESS_clean'] = df_MLS['MLS_ADDRESS_clean'].apply(lambda x: re.sub(' +', ' ', x))
df_DATATREE['DATATREE_ADDRESS_clean'] = df_DATATREE['DATATREE_ADDRESS_clean'].apply(lambda x: re.sub(' +', ' ', x))

In [ ]:
import usaddress
df1_MLS = df_MLS
def get_MLS_addr(df):
    global MLS_address
    try:
        MLS_address = usaddress.tag(df['MLS_ADDRESS_clean'])
    except usaddress.RepeatedLabelError as e :
#         print('error')
        pass
    return MLS_address

df1_MLS['MLS_ADDRESS_Split'] = df1_MLS.apply(lambda df1_MLS : get_MLS_addr(df1_MLS), axis = 1)
test_MLS = df1_MLS.MLS_ADDRESS_Split.to_list()
values= []
for i in range(len(test_MLS)):
    values.append(test_MLS[i][0])
splited_MLS = pd.DataFrame(values)
df1_MLS.reset_index(inplace=True)
MLS_ADDRESS_final = pd.concat([df1_MLS,splited_MLS],axis=1)
del MLS_ADDRESS_final['index']
MLS_ADDRESS_final

In [ ]:
# deal with datatree
df1_DATATREE = df_DATATREE
def get_DATATREE_addr(df):
    global DT_ADDRESS
    try:
        DT_ADDRESS = usaddress.tag(df['DATATREE_ADDRESS_clean'])
    except usaddress.RepeatedLabelError as e:
        pass
    return DT_ADDRESS
df1_DATATREE['DATATREE_ADDRESS_Split'] = df1_DATATREE.apply(lambda df1_DATATREE : get_DATATREE_addr(df1_DATATREE), axis = 1)
test_DATATREE = df1_DATATREE.DATATREE_ADDRESS_Split.to_list()
values= []
for i in range(len(test_DATATREE)):
    try:
        values.append(test_DATATREE[i][0])
    except:
        pass
splited_DATATREE = pd.DataFrame(values)
df1_DATATREE.reset_index(inplace=True)
DT_ADDRESS_final = pd.concat([df1_DATATREE,splited_DATATREE],axis=1)
del DT_ADDRESS_final['index']
DT_ADDRESS_final

In [ ]:
MLS_more_than_2 = MLS_ADDRESS_final[MLS_ADDRESS_final.groupby(['AddressNumber', 'StreetNamePreDirectional', 'StreetName', 'StreetNamePostType'])['CC_PROPERTY_ADDR_DISPLAY_1'].transform('count') > 2]
DATATREE_more_than_2 = DT_ADDRESS_final[DT_ADDRESS_final.groupby(['AddressNumber', 'StreetNamePreDirectional', 'StreetName', 'StreetNamePostType'])['SITUSFULLSTREETADDRESS'].transform('count') > 2]

In [ ]:
MLS_more_than_2 = MLS_more_than_2.sample(frac=1)
MLS_more_than_2_sample = pd.concat(
    [g.head(3) for _, g in MLS_more_than_2.groupby(['AddressNumber', 'StreetNamePreDirectional', 'StreetName', 'StreetNamePostType'], sort=False, as_index=False)][:500],
    ignore_index=True 
)
MLS_more_than_2_sample 

In [ ]:
DATATREE_more_than_2 = DATATREE_more_than_2.sample(frac=1)
DATATREE_more_than_2_sample = pd.concat(
    [g.head(3) for _, g in DATATREE_more_than_2.groupby(['AddressNumber', 'StreetNamePreDirectional', 'StreetName', 'StreetNamePostType'], sort=False, as_index=False)][:500],
    ignore_index=True 
)
DATATREE_more_than_2_sample 

In [ ]:
from placekey.api import PlacekeyAPI
from ast import literal_eval
placekey_api_key = "n1w1KOroOdSz3BVMMlg8IM4PuflcVP4V" # fill this in with your personal API key (do not share publicly)
pk_api = PlacekeyAPI(placekey_api_key)
#adding required placekey api field
def get_standard_df(df,column_map: dict):
    # rename column name --> need to be consistant with standatdized api field name
    # see this link: https://docs.placekey.io/#350ed3a9-68db-4c47-9e20-19b430cb9ef1
    df_standard = df.rename(columns=column_map)
    
    # adding other fields
    df_standard['query_id'] = range(1, 1+len(df))
    df_standard['iso_country_code'] = "US"
    return df_standard

def get_placekey_df(stad_df, place_keys:tuple):
    # stad_df is the df containing standatdized api fields
    import json
    # convert query_id to str
    stad_df = stad_df.astype({"query_id": str})
    
    # dump df to json-format with selected key-value pairs
    data_jsoned = json.loads(stad_df.to_json(orient="records"))
    dict_filter = lambda x, y: dict([ (i,x[i]) for i in x if i in set(y) ])
    result_dict=[dict_filter(i, place_keys) for i in data_jsoned]
    
    #request placekey api
    responses = pk_api.lookup_placekeys(result_dict, verbose=True)
    df_placekeys = pd.read_json(json.dumps(responses), dtype={'query_id':str})
    
    # TODO: need to handle missing / errors

    return df_placekeys

In [ ]:
column_map = {"CC_PROPERTY_ADDR_DISPLAY_1" : "street_address", 
              "CC_PROPERTY_ADDR_POSTAL_CODE": "postal_code",
              "CC_PROPERTY_ADDR_STATE": "region",
              "CC_PROPERTY_ADDR_CITY": "city"}
stad_df = get_standard_df(MLS_more_than_2_sample,column_map)
stad_df.reset_index(inplace=True)

place_keys = ("city","iso_country_code","postal_code","street_address","query_id","region")
df_placekey = get_placekey_df(stad_df,place_keys)

df_MLS_sample_placekey = pd.concat([df_placekey,MLS_more_than_2_sample],axis=1)

In [ ]:
df_MLS = df_MLS_sample_placekey.groupby(['AddressNumber', 'StreetNamePreDirectional', 'StreetName', 'StreetNamePostType'])
  
# call agg method
df_MLS = df_MLS.agg({"placekey": "nunique"})
  
# call reset_index method
df_MLS = df_MLS.reset_index()

In [ ]:
column_map = {"SITUSFULLSTREETADDRESS" : "street_address", 
              "SITUSZIP5": "postal_code",
              "SITUSSTATE": "region",
              "SITUSCITY": "city"}
stad_df = get_standard_df(DATATREE_more_than_2_sample,column_map)
stad_df.reset_index(inplace=True)

place_keys = ("city","iso_country_code","postal_code","street_address","query_id","region")
df_placekey = get_placekey_df(stad_df,place_keys)

df_DATATREE_sample_placekey = pd.concat([df_placekey,DATATREE_more_than_2_sample],axis=1)

In [ ]:
df_DATATREE = df_DATATREE_sample_placekey.groupby(['AddressNumber', 'StreetNamePreDirectional', 'StreetName', 'StreetNamePostType'])
  
# call agg method
df_DATATREE = df_DATATREE.agg({"placekey": "nunique"})
  
# call reset_index method
df_DATATREE = df_DATATREE.reset_index()

- placekey = 0 -> invalid address
- placekey = 1 -> same address as building address
- placekey = 2 -> some of the units have unique placekey while others do not
- placekey = 3 -> could identify different units

In [ ]:
df_MLS.groupby('placekey').size()/500

In [ ]:
df_DATATREE.groupby('placekey').size()/500